In [ ]:
# Importing Necessary Libraries
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = "0"
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import regularizers
from tensorflow.keras.utils import register_keras_serializable
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Reshape, GRU, BatchNormalization, Softmax


In [ ]:
# Mounting drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Importing helper function files
import sys
sys.path.append('/content/drive/MyDrive/CS 230 Project')
from utilLoss import *

In [ ]:
# Loading the model to calculate the response spectrum
responseModel = keras.models.load_model("/content/drive/MyDrive/CS 230 Project/responseModel.keras")
responseModel.trainable=False

In [ ]:
#Custom loss model
@register_keras_serializable()
def responseLoss(y,yhat):
    #MSE of predicted response spectra
    ySpectrum=responseModel(y)
    yhatSpectrum=responseModel(yhat)
    spectrumLoss=tf.divide(tf.reduce_mean(tf.square(tf.subtract(ySpectrum,yhatSpectrum))),tf.reduce_mean(tf.square(ySpectrum)))

    #Relative difference in Arias intensities
    aly=tf.reduce_sum(tf.square(y))
    alyhat=tf.reduce_sum(tf.square(yhat))
    ariasLoss=tf.divide(tf.abs(tf.subtract(aly,alyhat)),aly)

    #Direct MSE comparison of records, normalized
    motionLoss=tf.divide(tf.reduce_mean(tf.square(tf.subtract(y,yhat))),tf.reduce_mean(tf.square(y)))

    #Equally weight normalized losses
    return tf.divide(tf.add(spectrumLoss,tf.add(ariasLoss,motionLoss)),3.0)

In [ ]:
# Loading the Preprocessed Data
tf.compat.v1.enable_eager_execution()
xfilepath="/content/drive/MyDrive/CS 230 Project/4096_dataset/inputExpanded.csv"
ypath="/content/drive/MyDrive/CS 230 Project/4096_dataset/output.npy"

input_data= pd.read_csv(xfilepath)
x= input_data.to_numpy()
y=np.load(ypath)

#Shuffle and split data into Train and Test Sets
m=x.shape[0]
testSplit=0.05
shuffle=np.arange(m)
np.random.shuffle(shuffle)
x=x[shuffle,:]
y=y[shuffle,:]
index=int(np.floor(m*(1-testSplit)))
xTrain=x[0:index,:]
yTrain=y[0:index,:]
xTest=x[index:,:]
yTest=y[index:,:]

In [ ]:
print(x.shape, y.shape, xTrain.shape, yTrain.shape, xTest.shape, yTest.shape)

(53959, 93) (53959, 4096) (51261, 93) (51261, 4096) (2698, 93) (2698, 4096)


In [ ]:
# Model Desc: 1 Dense + 2 LSTM Layers + MSE Error -- Baseline

# Constants
INPUT_FEATURES = xTrain.shape[1]       # Number of input features
SEQUENCE_LENGTH = yTrain.shape[1]      # Length of the output sequence
HIDDEN_UNITS_1 = 1024        # LSTM layer 1 units
HIDDEN_UNITS_2 = 1024        # LSTM layer 2 units
DENSE_UNITS = 1024          # Dense layer units

# Model Definition
def build_model(input_features, sequence_length, dense_units, hidden_units_1, hidden_units_2):
    # Input Layer
    input_layer = Input(shape=(input_features,))

    # Dense Layer
    dense = Dense(dense_units, activation='relu')(input_layer)

    # Reshape Layer (to prepare for LSTM input)
    reshape = Reshape((1, dense_units))(dense)

    # LSTM Layers
    lstm_1 = LSTM(hidden_units_1, return_sequences=True)(reshape)
    lstm_2 = LSTM(hidden_units_2, return_sequences=True)(lstm_1)

    # Output Layer (Dense with linear activation)
    output_layer = Dense(sequence_length, activation='linear')(lstm_2)

    # Create Model
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mse')
    return model

# Build and compile the model
model = build_model(INPUT_FEATURES, SEQUENCE_LENGTH, DENSE_UNITS, HIDDEN_UNITS_1, HIDDEN_UNITS_2)

# Summary of the model
model.summary()
# Train the model
model.fit(xTrain, yTrain, epochs=1, batch_size=1024, validation_split=0.1)
# Save the model
model.save('/content/drive/MyDrive/CS 230 Project/lstm_model_1')

# Load the model
from tensorflow.keras.models import load_model
loaded_model = load_model('/content/drive/MyDrive/CS 230 Project/lstm_model_1')

# Retrain the loaded model

history = loaded_model.fit(xTrain, yTrain, epochs=10, batch_size=1024, validation_split=0.1)

# Plot training & validation loss values
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Save the model
model.save('/content/drive/MyDrive/CS 230 Project/lstm_model_1')

In [ ]:
# Model Desc: 1 Dense + 2 LSTM Layers + MSE Error + LR = 0.01

# Constants
INPUT_FEATURES = xTrain.shape[1]       # Number of input features
SEQUENCE_LENGTH = yTrain.shape[1]      # Length of the output sequence
HIDDEN_UNITS_1 = 1024        # LSTM layer 1 units
HIDDEN_UNITS_2 = 1024        # LSTM layer 2 units
DENSE_UNITS = 1024          # Dense layer units

# Model Definition
def build_model(input_features, sequence_length, dense_units, hidden_units_1, hidden_units_2):
    # Input Layer
    input_layer = Input(shape=(input_features,))

    # Dense Layer
    dense = Dense(dense_units, activation='relu')(input_layer)

    # Reshape Layer (to prepare for LSTM input)
    reshape = Reshape((1, dense_units))(dense)

    # LSTM Layers
    lstm_1 = LSTM(hidden_units_1, return_sequences=True)(reshape)
    lstm_2 = LSTM(hidden_units_2, return_sequences=True)(lstm_1)

    # Output Layer (Dense with linear activation)
    output_layer = Dense(sequence_length, activation='linear')(lstm_2)

    # Create Model
    model = Model(inputs=input_layer, outputs=output_layer)
    optimizer = keras.optimizers.Adam(learning_rate=0.01)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Build and compile the model
model_lrplus = build_model(INPUT_FEATURES, SEQUENCE_LENGTH, DENSE_UNITS, HIDDEN_UNITS_1, HIDDEN_UNITS_2)

# Summary of the model
model_lrplus.summary()
# Train the model
model_lrplus.fit(xTrain, yTrain, epochs=5, batch_size=64, validation_split=0.1)

In [ ]:
# Model Desc: 1 Dense + 2 LSTM Layers + MSE Error + LR = 0.0001

# Constants
INPUT_FEATURES = xTrain.shape[1]       # Number of input features
SEQUENCE_LENGTH = yTrain.shape[1]      # Length of the output sequence
HIDDEN_UNITS_1 = 1024        # LSTM layer 1 units
HIDDEN_UNITS_2 = 1024        # LSTM layer 2 units
DENSE_UNITS = 1024          # Dense layer units

# Model Definition
def build_model(input_features, sequence_length, dense_units, hidden_units_1, hidden_units_2):
    # Input Layer
    input_layer = Input(shape=(input_features,))

    # Dense Layer
    dense = Dense(dense_units, activation='relu')(input_layer)

    # Reshape Layer (to prepare for LSTM input)
    reshape = Reshape((1, dense_units))(dense)

    # LSTM Layers
    lstm_1 = LSTM(hidden_units_1, return_sequences=True)(reshape)
    lstm_2 = LSTM(hidden_units_2, return_sequences=True)(lstm_1)

    # Output Layer (Dense with linear activation)
    output_layer = Dense(sequence_length, activation='linear')(lstm_2)

    # Create Model
    model = Model(inputs=input_layer, outputs=output_layer)
    optimizer = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Build and compile the model
model_lrminus = build_model(INPUT_FEATURES, SEQUENCE_LENGTH, DENSE_UNITS, HIDDEN_UNITS_1, HIDDEN_UNITS_2)

# Summary of the model
model_lrminus.summary()
# Train the model
history_minus = model_lrminus.fit(xTrain, yTrain, epochs=5, batch_size=64, validation_split=0.1)
# Save the model
model_lrminus.save('/content/drive/MyDrive/Colab Notebooks/rnn_model_1024_lrminus.keras')

In [ ]:
# Model Desc: 2 Dense + 2 LSTM Layers + MSE Error + LR = 0.0001 (Extra Dense Layer)
# Constants
INPUT_FEATURES = xTrain.shape[1]       # Number of input features
SEQUENCE_LENGTH = yTrain.shape[1]      # Length of the output sequence
HIDDEN_UNITS_1 = 1024        # LSTM layer 1 units
HIDDEN_UNITS_2 = 1024        # LSTM layer 2 units
DENSE_UNITS = 1024          # Dense layer units

# Model Definition
def build_model(input_features, sequence_length, dense_units, hidden_units_1, hidden_units_2):
    # Input Layer
    input_layer = Input(shape=(input_features,))

    # Dense Layer
    dense = Dense(dense_units, activation='relu')(input_layer)

    # Reshape Layer (to prepare for LSTM input)
    reshape = Reshape((1, dense_units))(dense)

    # LSTM Layers
    lstm_1 = LSTM(hidden_units_1, return_sequences=True)(reshape)
    lstm_2 = LSTM(hidden_units_2, return_sequences=True)(lstm_1)

    # Output Layer (Dense with linear activation)
    output_layer = Dense(sequence_length, activation='linear')(lstm_2)

    # Create Model
    model = Model(inputs=input_layer, outputs=output_layer)
    optimizer = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Build and compile the model
model = build_model(INPUT_FEATURES, SEQUENCE_LENGTH, DENSE_UNITS, HIDDEN_UNITS_1, HIDDEN_UNITS_2)

# Summary of the model
model.summary()
# Train the model
model.fit(xTrain, yTrain, epochs=1, batch_size=64, validation_split=0.1)
# Save the model
model.save('/content/drive/MyDrive/CS 230 Project/lstm_model_2')

# Load the model
from tensorflow.keras.models import load_model
loaded_model = load_model('/content/drive/MyDrive/CS 230 Project/lstm_model_2')

# Retrain the loaded model
loaded_model.fit(xTrain, yTrain, epochs=10, batch_size=64, validation_split=0.1)
# Save the model
model.save('/content/drive/MyDrive/CS 230 Project/lstm_model_2')

In [ ]:
# Model Desc: 2 Dense + 2 LSTM Layers + + LR = 0.0001 +  Response Loss -- Custom Loss Function
# Constants
INPUT_FEATURES = xTrain.shape[1]       # Number of input features
SEQUENCE_LENGTH = yTrain.shape[1]      # Length of the output sequence
HIDDEN_UNITS_1 = 1024        # LSTM layer 1 units
HIDDEN_UNITS_2 = 1024        # LSTM layer 2 units
DENSE_UNITS = 1024          # Dense layer units
DENSE_UNITS_1 = 256         # Extra Dense layer units

# Model Definition
def build_model(input_features, sequence_length, extradense_units, dense_units, hidden_units_1, hidden_units_2):
    # Input Layer
    input_layer = Input(shape=(input_features,))

    # Dense Layer
    dense = Dense(extradense_units, activation='relu')(input_layer)
    dense = Dense(dense_units, activation='relu')(dense)

    # Reshape Layer (to prepare for LSTM input)
    reshape = Reshape((1, dense_units))(dense)

    # LSTM Layers
    lstm_1 = LSTM(hidden_units_1, return_sequences=True)(reshape)
    lstm_2 = LSTM(hidden_units_2, return_sequences=True)(lstm_1)

    # Output Layer (Dense with linear activation)
    output_layer = Dense(sequence_length, activation='linear')(lstm_2)

    # Create Model
    model = Model(inputs=input_layer, outputs=output_layer)
    optimizer = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss=responseLoss, metrics=[responseLoss])
    return model

# Build and compile the model
model_extradense = build_model(INPUT_FEATURES, SEQUENCE_LENGTH,DENSE_UNITS_1, DENSE_UNITS, HIDDEN_UNITS_1, HIDDEN_UNITS_2)

# Summary of the model
model_extradense.summary()


Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 93)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │          24,064 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1024)                │         263,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 1, 1024)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 1, 1024)             │       8,392,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 1, 1024)             │       8,392,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1, 4096)             │       4,198,400 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,271,040 (81.14 MB)

 Trainable params: 21,271,040 (81.14 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
history_minus = model_extradense.fit(xTrain, yTrain, epochs=1, batch_size=64, validation_split=0.1)

721/721 ━━━━━━━━━━━━━━━━━━━━ 1370s 2s/step - loss: 0.9397 - response_loss: 0.9397 - val_loss: 0.8170 - val_response_loss: 0.8162


In [ ]:
# Save the model
model_extradense.save('/content/drive/MyDrive/CS 230 Project/rnn_model_1024_extradense_response_loss.keras')

In [ ]:
# Model Desc: 2 Dense + 2 LSTM Layers + Response Loss + Learning Rate = 0.0001 + Batch Normalization --- Final Model
def build_model(input_features, sequence_length, extradense_units, dense_units, hidden_units_1, hidden_units_2):
    # Input Layer
    input_layer = Input(shape=(input_features,))

    # Dense Layer + Batch Normalization
    dense = Dense(extradense_units, activation='relu')(input_layer)
    dense = BatchNormalization()(dense)  # Batch Normalization after first Dense
    dense = Dense(dense_units, activation='relu')(dense)
    dense = BatchNormalization()(dense)  # Batch Normalization after second Dense

    # Reshape Layer (to prepare for LSTM input)
    reshape = Reshape((1, dense_units))(dense)

    # LSTM Layers
    lstm_1 = LSTM(hidden_units_1, return_sequences=True)(reshape)
    lstm_2 = LSTM(hidden_units_2, return_sequences=True)(lstm_1)

    # Output Layer (Dense with linear activation)
    output_layer = Dense(sequence_length, activation='linear')(lstm_2)

    # Create Model
    model = Model(inputs=input_layer, outputs=output_layer)
    optimizer = keras.optimizers.Adam(learning_rate=0.0001)

    # Compile the model with custom loss function
    model.compile(optimizer=optimizer, loss=responseLoss, metrics=[responseLoss])

    return model
model_1024_batch_norm=build_model(INPUT_FEATURES, SEQUENCE_LENGTH,DENSE_UNITS_1, DENSE_UNITS, HIDDEN_UNITS_1, HIDDEN_UNITS_2)

# Summary of the model
model_1024_batch_norm.summary()

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 93)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 256)                 │          24,064 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1024)                │         263,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 1024)                │           4,096 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_1 (Reshape)                  │ (None, 1, 1024)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 1, 1024)             │       8,392,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 1, 1024)             │       8,392,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1, 4096)             │       4,198,400 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,276,160 (81.16 MB)

 Trainable params: 21,273,600 (81.15 MB)

 Non-trainable params: 2,560 (10.00 KB)

In [ ]:
# Train the model
history_1024_batch_norm = model_1024_batch_norm.fit(xTrain, yTrain, epochs=1, batch_size=64, validation_split=0.1)

721/721 ━━━━━━━━━━━━━━━━━━━━ 1362s 2s/step - loss: 0.8392 - response_loss: 0.8392 - val_loss: 0.8040 - val_response_loss: 0.8033


In [ ]:
# Save the model
model_1024_batch_norm.save('/content/drive/MyDrive/CS 230 Project/rnn_model_1024_batchnorm_response_loss.keras')

In [ ]:
# Load and retrain the model
model_1024_batch_norm = keras.models.load_model("/content/drive/MyDrive/CS 230 Project/rnn_model_1024_batchnorm_response_loss.keras", custom_objects={'responseLoss': responseLoss})
history_1024_batch_norm = model_1024_batch_norm.fit(xTrain, yTrain, epochs=2, batch_size=32, validation_split=0.1)


Epoch 1/2
1442/1442 ━━━━━━━━━━━━━━━━━━━━ 1414s 977ms/step - loss: 0.8312 - response_loss: 0.8312 - val_loss: 0.8274 - val_response_loss: 0.8279
Epoch 2/2
1442/1442 ━━━━━━━━━━━━━━━━━━━━ 1451s 970ms/step - loss: 0.8074 - response_loss: 0.8074 - val_loss: 0.8027 - val_response_loss: 0.8034


In [ ]:
model_1024_batch_norm.save('/content/drive/MyDrive/CS 230 Project/rnn_model_1024_batchnorm_response_loss.keras')

In [ ]:
# Plot to compare the generated and actual sequence
%matplotlib inline

# Number of examples to plot
num_examples = 8

# Create a figure and subplots
fig, axes = plt.subplots(num_examples, 2, figsize=(12, 4 * num_examples))

# Iterate through the examples
for i in range(num_examples):
    # Select a random example from the test set
    example_index = np.random.randint(0, len(xTest))
    test_input = np.reshape(xTest[example_index, :], (1, 93))
    generated_sequence = model_1024_batch_norm.predict(test_input)

    # Plot the generated sequence
    axes[i, 0].plot(generated_sequence[0, 0, :], label='Generated Sequence', marker='o')
    axes[i, 0].set_title(f"Generated Sequence (Example {i+1})")
    axes[i, 0].set_xlabel("Time Step")
    axes[i, 0].set_ylabel("Value")
    axes[i, 0].legend()

    # Plot the true sequence
    axes[i, 1].plot(yTest[example_index, :], label='True Sequence', marker='x')
    axes[i, 1].set_title(f"True Sequence (Example {i+1})")
    axes[i, 1].set_xlabel("Time Step")
    axes[i, 1].set_ylabel("Value")
    axes[i, 1].legend()

# Adjust spacing between subplots
plt.tight_layout()
plt.show()

In [ ]:
# Model Desc: 2 Dense + 2 GRU Layers + Response Loss + Learning Rate = 0.0001 + Batch Normalization

def build_model(input_features, sequence_length, extradense_units, dense_units, hidden_units_1, hidden_units_2):
    # Input Layer
    input_layer = Input(shape=(input_features,))


    # Dense Layer + Batch Normalization
    dense = Dense(extradense_units, activation='relu')(input_layer)
    dense = BatchNormalization()(dense)  # Batch Normalization after the first Dense
    dense = Dense(dense_units, activation='relu')(dense)
    dense = BatchNormalization()(dense)  # Batch Normalization after the second Dense

    # Reshape Layer (to prepare for GRU input)
    reshape = Reshape((1, dense_units))(dense)

    # GRU Layers
    gru_1 = GRU(hidden_units_1, return_sequences=True)(reshape)
    gru_2 = GRU(hidden_units_2, return_sequences=True)(gru_1)

    # Output Layer (Dense with linear activation)
    output_layer = Dense(sequence_length, activation='linear')(gru_2)

    # Create Model
    model = Model(inputs=input_layer, outputs=output_layer)
    optimizer = keras.optimizers.Adam(learning_rate=0.0001)

    # Compile the model with custom loss function
    model.compile(optimizer=optimizer, loss=responseLoss, metrics=[responseLoss])

    return model

model_1024_batch_norm_gru=build_model(INPUT_FEATURES, SEQUENCE_LENGTH,DENSE_UNITS_1, DENSE_UNITS, HIDDEN_UNITS_1, HIDDEN_UNITS_2)

# Summary of the model
model_1024_batch_norm_gru.summary()

Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 93)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 256)                 │          24,064 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 1024)                │         263,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 1024)                │           4,096 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_4 (Reshape)                  │ (None, 1, 1024)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_4 (GRU)                          │ (None, 1, 1024)             │       6,297,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_5 (GRU)                          │ (None, 1, 1024)             │       6,297,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 1, 4096)             │       4,198,400 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 17,085,952 (65.18 MB)

 Trainable params: 17,083,392 (65.17 MB)

 Non-trainable params: 2,560 (10.00 KB)

In [ ]:
# Train the model
history_1024_batch_norm = model_1024_batch_norm_gru.fit(xTrain, yTrain, epochs=1, batch_size=32, validation_split=0.1)

In [ ]:
# Save the model
model_1024_batch_norm_gru.save('/content/drive/MyDrive/CS 230 Project/rnn_model_1024_batchnorm_response_loss_gru.keras')